In [1]:
import tensorflow as tf
 
# The file path to save the data
save_file = './model.ckpt'
 
# Two Tensor Variables: weights and bias
weights = tf.Variable(tf.truncated_normal([2, 3]))
bias = tf.Variable(tf.truncated_normal([3]))
 
# Class used to save and/or restore Tensor Variables
saver = tf.train.Saver()
 
with tf.Session() as sess:
    # Initialize all the Variables
    sess.run(tf.global_variables_initializer())
 
    # Show the values of weights and bias
    print('Weights:')
    print(sess.run(weights))
    print('Bias:')
    print(sess.run(bias))
 
    # Save the model
    saver.save(sess, save_file)

AttributeError: module 'tensorflow' has no attribute 'truncated_normal'

In [ ]:
# Remove the previous weights and bias
tf.reset_default_graph()

# Two Variables: weights and bias
weights = tf.Variable(tf.truncated_normal([2, 3]))
bias = tf.Variable(tf.truncated_normal([3]))
 
# Class used to save and/or restore Tensor Variables
saver = tf.train.Saver()
 
with tf.Session() as sess:
    # Load the weights and bias
    saver.restore(sess, save_file)
 
    # Show the values of weights and bias
    print('Weight:')
    print(sess.run(weights))
    print('Bias:')
    print(sess.run(bias))

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
 
learning_rate = 0.001
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)
 
# Import MNIST data
mnist = input_data.read_data_sets('.', one_hot=True)
 
# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])
 
# Weights & bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))
 
# Logits - xW + b
logits = tf.add(tf.matmul(features, weights), bias)
 
# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)
 
# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
 
 
import math
save_file = './train_model.ckpt'
batch_size = 128
n_epochs = 100
 
saver = tf.train.Saver()
 
# Launch the graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
 
    # Training cycle
    for epoch in range(n_epochs):
        total_batch = math.ceil(mnist.train.num_examples / batch_size)
 
        # Loop over all batches
        for i in range(total_batch):
            batch_features, batch_labels = mnist.train.next_batch(batch_size)
            sess.run(
                optimizer,
                feed_dict={features: batch_features, labels: batch_labels})
 
        # Print status for every 10 epochs
        if epoch % 10 == 0:
            valid_accuracy = sess.run(
                accuracy,
                feed_dict={
                    features: mnist.validation.images,
                    labels: mnist.validation.labels})
            print('Epoch {:<3} - Validation Accuracy: {}'.format(
                epoch,
                valid_accuracy))
 
    # Save the model
    saver.save(sess, save_file)
    print('Trained Model Saved.')

In [3]:
v = tf.Variable(0.0)
v.assign_add(1)
v.read_value()  # 1.0

<tf.Tensor: id=18, shape=(), dtype=float32, numpy=1.0>

In [4]:
class MyModuleOne(tf.Module):
  def __init__(self):
    self.v0 = tf.Variable(1.0)
    self.vs = [tf.Variable(x) for x in range(10)]
    
class MyOtherModule(tf.Module):
  def __init__(self):
    self.m = MyModuleOne()
    self.v = tf.Variable(10.0)
    
m = MyOtherModule()
len(m.variables)  # 12; 11은 m.m에서 다른 것은 m.v에서

12

In [ ]:
weights = tf.placeholder(name="loss_weights", shape=[None], dtype=tf.float32)
loss_per_example = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels)
loss = tf.reduce_mean(weights * loss_per_example)

In [ ]:
def _model_def(self):
    # create placeholder variables
    self.input_images = tf.placeholder(tf.float32, shape=[None, self._input_size])
    self.labels = tf.placeholder(tf.float32, shape=[None, self._label_size])
    # create self._num_layers dense layers as the model
    input = self.input_images
    tf.summary.scalar("input_var", self._calculate_variance(input))
    for i in range(self._num_layers - 1):
        input = tf.layers.dense(input, self._hidden_size, kernel_initializer=self._init,
                                activation=self._activation, name='layer{}'.format(i+1))
        # get the input to the nodes (sans bias)
        mat_mul_in = tf.get_default_graph().get_tensor_by_name("layer{}/MatMul:0".format(i + 1))
        # log pre and post activation function histograms
        tf.summary.histogram("mat_mul_hist_{}".format(i + 1), mat_mul_in)
        tf.summary.histogram("fc_out_{}".format(i + 1), input)
        # also log the variance of mat mul
        tf.summary.scalar("mat_mul_var_{}".format(i + 1), self._calculate_variance(mat_mul_in))
    # don't supply an activation for the final layer - the loss definition will
    # supply softmax activation. This defaults to a linear activation i.e. f(x) = x
    logits = tf.layers.dense(input, 10, name='layer{}'.format(self._num_layers))
    mat_mul_in = tf.get_default_graph().get_tensor_by_name("layer{}/MatMul:0".format(self._num_layers))
    tf.summary.histogram("mat_mul_hist_{}".format(self._num_layers), mat_mul_in)
    tf.summary.histogram("fc_out_{}".format(self._num_layers), input)
    # use softmax cross entropy with logits - no need to apply softmax activation to
    # logits
    self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                                         labels=self.labels))
    # add the loss to the summary
    tf.summary.scalar('loss', self.loss)
    self.optimizer = tf.train.AdamOptimizer().minimize(self.loss)
    self.accuracy = self._compute_accuracy(logits, self.labels)
    tf.summary.scalar('acc', self.accuracy)
    self.merged = tf.summary.merge_all()
    self.init_op = tf.global_variables_initializer()